In [1]:
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from joblib import dump, load

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
import skimage.data as data
import skimage.color as color
import skimage
import skimage.measure as measure
from skimage.morphology import square
import skimage.morphology
import sys
import json


d = {}

def process_directory(directory):
    """
    Process dricetory - dir with images is nested, there is main dir and within there are dirs with leafs species.
    This function saves all species dirs from main dir to list.
    
    Parameters
    ----------
    directory : str
        Main directory to be processed.
    
    Returns
    -------
    dirs
        List of species dirs.
    """
    dirs = []
    for dir in os.listdir(directory):
        if os.path.isdir(os.path.join(directory, dir)):
            dirs.append(dir)
    
    return dirs


def preprocess_img(img):
    """
    Makes images preprocessing:
        - changes images to black and white spectrum,
        - removes noise,
        - makes image dilatation.

    Parameters
    ----------
    img : skimage.io object
        Image to be preprocessed.
    
    Returns
    -------
    mask
        Image mask used in the next stage.
    """
    imgg = color.rgb2gray(img)
    mask = (imgg < 0.5)
    mask = skimage.morphology.dilation(mask, square(3))
    return mask


def get_leaf(mask):
    """
    Extracts leaf from image.
    
    Parameters
    ----------
    mask : np.array
        Image from which leaf will be extracted.
    
    Returns
    -------
    numpy.array : leaf.
    int : leaf's area.
    int : number of pixels of convex hull image, which is the smallest convex polygon that encloses the region.
    int : the diameter of a circle with the same area as the region.
    """
    labels = measure.label(mask)
    props = skimage.measure.regionprops(labels)
    areas = np.array([(i, props[i]['area']) for i in range(1, labels.max()) if props[i]['area'] > 1000])
    minimal = 0
    pos = 0
    for pair in areas:
        distance = sum(props[pair[0]]['centroid']) - 800
        if minimal == 0 or distance < minimal:
            minimal = distance
            pos = pair[0]

    return props[pos]['image'], props[pos]['area'], props[pos]['convex_area'], np.rint(props[pos]['equivalent_diameter'])


def calculate_tail_area(img):
    """
    Calculates area of leaf's tail.
    
    Parameters
    ----------
    img : np.array
        Numpy array of image from which tail area will be calculated.
    
    Returns
    -------
    int : Tail's area of given leaf.
    """
    imgg = skimage.morphology.erosion(img, square(5))
    imgg = img ^ imgg
    imgg = skimage.morphology.binary_opening(imgg, square(3))
    
    tmp_labels = measure.label(imgg)
    tmp_props = skimage.measure.regionprops(tmp_labels)
    try:
        return int(max([tmp_props[i]['area'] for i in range(0, tmp_labels.max())]))
    except Exception:
        return 0


def calculate_perimeter(img):
    """
    Calculates leaf perimeter.
    
    Parameters
    ----------
    img : np.array
        Numpy array of image which perimeter will be calculated.
    
    Returns
    -------
    int : Perimeter of leaf.
    """
    img = skimage.morphology.binary_opening(img, square(5))
    tmp_labels = measure.label(img)
    tmp_props = skimage.measure.regionprops(tmp_labels)
    return int(sum([tmp_props[i]['perimeter'] for i in range(0, tmp_labels.max())]))


def calculate_skeleton(img):
    """
    Calculates leaf's skeleton.
    
    Parameters
    ----------
    img : np.array
        Numpy array of image which skeleton will be calculated.
    
    Returns
    -------
    int : Sum of pixels in leaf's skeleton.
    """
    return int(np.sum(skimage.morphology.skeletonize(img)))


def calculate_major_axis_length(img):
    """
    Calculates the length of major axis of leaf.
    
    Parameters
    ----------
    img : np.array
        Numpy array of image which major axis will be calculated.
    
    Returns
    -------
    int : Leaf's major axis length.
    """
    img = skimage.morphology.binary_opening(img, square(5))
    tmp_labels = measure.label(img)
    tmp_props = skimage.measure.regionprops(tmp_labels)
    return int(max([tmp_props[i]['major_axis_length'] for i in range(0, tmp_labels.max())]))


def calculate_minor_axis_length(img):
    """
    Calculates the length of minor axis of leaf.
    
    Parameters
    ----------
    img : np.array
        Numpy array of image which minor axis will be calculated.
    
    Returns
    -------
    int : Leaf's minor axis length.
    """
    img = skimage.morphology.binary_opening(img, square(5))
    tmp_labels = measure.label(img)
    tmp_props = skimage.measure.regionprops(tmp_labels)
    return int(max([tmp_props[i]['minor_axis_length'] for i in range(0, tmp_labels.max())]))


def calculate_mean_distance_from_center(img):
    """
    Calculates the mean distance of each contour point from the center of the leaf.
    
    Parameters
    ----------
    img : np.array
        Numpy array of image which mean distance of all contour points will be calculated.
    
    Returns
    -------
    int : Mean distance of all contour points from the center of the leaf.
    """
    img = skimage.morphology.binary_opening(img, square(5))
    contour = skimage.measure.find_contours(img)
    
    labels = measure.label(img)
    props = skimage.measure.regionprops(labels)

    areas = np.array([(i, props[i]['area']) for i in range(1, labels.max()) if props[i]['area'] > 1000])
    maximal = 0
    pos = 0
    for pair in areas:
        if maximal == 0 or pair[1] > maximal:
            maximal = pair[1]
            pos = pair[0]
    
    center = np.array(props[pos]['centroid'])
    contour[0] -= center
    return int(np.rint(np.mean(contour[0])))


def save_to_dict(mask, dir, it):
    """
    Calls functions which calculates leaf's properties and saves results to dictionary.
    
    Parameters
    ----------
    mask : np.array
        Mask returned by preprocess_img function.
    dir : str
        Directory name with given species.
    it : int
        Image number.
    """
    img, area, convex_area, equivalent_diameter = get_leaf(mask)
    d[dir][it]['area'] = int(area)
    d[dir][it]['convex_area'] = int(convex_area)
    d[dir][it]['equivalent_diameter'] = int(equivalent_diameter)
    d[dir][it]['tail_area'] = calculate_tail_area(img)
    d[dir][it]['perimeter'] = calculate_perimeter(img)
    d[dir][it]['skeleton'] = calculate_skeleton(img)
    d[dir][it]['major_axis_length'] = calculate_major_axis_length(img)
    d[dir][it]['minor_axis_length'] = calculate_minor_axis_length(img)
    d[dir][it]['mean_distance_from_center'] = calculate_mean_distance_from_center(img)


def save_dict(file_name):
    """
    Saves results of images processing to file in JSON fomrat.

    Parameters
    ----------
    file_name : str
        Name of file to which data will be saved.
    """
    with open(file_name + '.json', 'w') as f:
        json.dump(d, f, indent=4)


def process_single_species(dir):
    """
    Calls preprocessing fuction for images in given directory and then calls save_to_dict function.

    Parameters
    ----------
    dir : str
        Name of directory which images will be processed.
    """
    for i in os.listdir():
        d[dir] = {}

    for it, file in enumerate(os.listdir()):
        d[dir][it] = {}
        img = io.imread(file)
        mask = preprocess_img(img)
        save_to_dict(mask, dir, it)


def process_files(main_dir, dirs_lst):
    """
    Process main directory and calls process_single_species function.
    
    Parameters
    ----------
    main_dir : str
        Name of main dir to be processed.
    dirs_lst : list
        List of directories with leafs species.
    """
    os.chdir(main_dir)

    for dir in dirs_lst:
        print('Processing: ' + dir)
        os.chdir(dir)
        process_single_species(dir)
        os.chdir('..')
        print('Done')
    
    os.chdir('..')


dirs = process_directory('leafsnap-subset1')
process_files('leafsnap-subset1', dirs)

Processing: acer_campestre
Done
Processing: acer_palmatum
Done
Processing: betula_alleghaniensis
Done
Processing: betula_jacqemontii
Done
Processing: carya_glabra
Done
Processing: celtis_occidentalis
Done
Processing: cercis_canadensis
Done
Processing: cornus_florida
Done
Processing: crataegus_crus-galli
Done
Processing: fagus_grandifolia
Done
Processing: ginkgo_biloba
Done
Processing: halesia_tetraptera
Done
Processing: ilex_opaca
Done
Processing: liquidambar_styraciflua
Done
Processing: magnolia_soulangiana
Done
Processing: malus_pumila
Done
Processing: morus_rubra
Done
Processing: ostrya_virginiana
Done
Processing: platanus_acerifolia
Done
Processing: populus_grandidentata
Done


In [4]:
knn = KNeighborsClassifier()
svm_clf = svm.SVC()
gnb = GaussianNB()
tree_clf = tree.DecisionTreeClassifier()
random_forest_clf = RandomForestClassifier()
sgdc_clf = SGDClassifier()

In [5]:
def create_data_array():
    """
    Creates data, target and target names arrays.
    
    Returns
    -------
    data : np.array
        Array of values for all calculated leafs properties.
    target : np.array
        Array of numbers which corresponds to leafs species.
    target_names : np.array
        Array of names of leafs species.
    """
    data = []
    target = []
    target_names = np.array([key for key in d.keys()])
    for pos, key in enumerate(d):
        for it in d[key]:
            tmp_data = []
            target.append(pos)
            for param in d[key][it]:
                tmp_data.append(d[key][it][param])
            data.append(np.array(tmp_data))
    
    data = np.array(data)
    target = np.array(target)
    
    return data, target, target_names

def count_mistakes(current, real):
    difference = current - real
    return len(difference[difference != 0])

In [6]:
data, target, target_names = create_data_array()

In [7]:
def calculate_mean_fit(data, target):
    """
    Calculates mean fit for all classificators.
    
    Parameters
    ----------
    data : np.array
        Array of values for all calculated leafs properties.
    target : np.array
        Array of numbers which corresponds to leafs species.
    """
    knn_lst = []
    svm_lst = []
    sgdc_lst = []
    gnb_lst = []
    tree_lst = []
    random_forest_lst = []

    for i in range(100):
        knn = KNeighborsClassifier()
        svm_clf = svm.SVC()
        gnb = GaussianNB()
        tree_clf = tree.DecisionTreeClassifier()
        random_forest_clf = RandomForestClassifier()
        sgdc_clf = SGDClassifier()

        indices = np.random.permutation(len(data))
        train_X = data[indices[:-100]]
        train_Y = target[indices[:-100]]
        test_X = data[indices[-100:]]
        test_Y = target[indices[-100:]]
        
        knn.fit(train_X, train_Y)
        svm_clf.fit(train_X, train_Y)
        sgdc_clf.fit(train_X, train_Y)
        gnb.fit(train_X, train_Y)
        tree_clf.fit(train_X, train_Y)
        random_forest_clf.fit(train_X, train_Y)
        
        knn_lst.append((len(test_X) - count_mistakes(knn.predict(test_X), test_Y)) / len(test_X))
        svm_lst.append((len(test_X) - count_mistakes(svm_clf.predict(test_X), test_Y)) / len(test_X))
        sgdc_lst.append((len(test_X) - count_mistakes(sgdc_clf.predict(test_X), test_Y)) / len(test_X))
        gnb_lst.append((len(test_X) - count_mistakes(gnb.predict(test_X), test_Y)) / len(test_X))
        tree_lst.append((len(test_X) - count_mistakes(tree_clf.predict(test_X), test_Y)) / len(test_X))
        random_forest_lst.append((len(test_X) - count_mistakes(random_forest_clf.predict(test_X), test_Y)) / len(test_X))
                                 
    print('Correct percent for KNN: ' + str(round(np.mean(knn_lst), 2)) + '%')
    print('Correct percent for SVM: ' + str(round(np.mean(svm_lst), 2)) + '%')
    print('Correct percent for SGDC: ' + str(round(np.mean(sgdc_lst), 2)) + '%')
    print('Correct percent for GNB: ' + str(round(np.mean(gnb_lst), 2)) + '%')
    print('Correct percent for TREE: ' + str(round(np.mean(tree_lst), 2)) + '%')
    print('Correct percent for RANDOM FOREST: ' + str(round(np.mean(random_forest_lst), 2)) + '%')

In [8]:
calculate_mean_fit(data, target)

Correct percent for KNN: 0.51%
Correct percent for SVM: 0.28%
Correct percent for SGDC: 0.09%
Correct percent for GNB: 0.44%
Correct percent for TREE: 0.65%
Correct percent for RANDOM FOREST: 0.78%


In [9]:
#################################################################### feature selection

In [10]:
data.shape

(800, 9)

In [11]:
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
low_variance_removed_data = sel.fit_transform(data)

In [12]:
low_variance_removed_data.shape

(800, 9)

In [13]:
univariate__feature_selection_data = SelectKBest(chi2, k=4).fit_transform(data, target) # chi2 test rquieres non-negative values

ValueError: Input X must be non-negative.

In [14]:
clf = ExtraTreesClassifier(n_estimators=150)
clf = clf.fit(data, target)
model = SelectFromModel(clf, prefit=True)
X_new = model.transform(data)
X_new.shape

(800, 4)

In [15]:
calculate_mean_fit(X_new, target)

Correct percent for KNN: 0.27%
Correct percent for SVM: 0.17%
Correct percent for SGDC: 0.06%
Correct percent for GNB: 0.37%
Correct percent for TREE: 0.64%
Correct percent for RANDOM FOREST: 0.74%


In [16]:
#################################################################### parameters tuning

In [17]:
random_forest_clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [18]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [19]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(data, target)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [20]:
rf_random.best_params_

{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 50,
 'bootstrap': False}

In [21]:
def evaluate(model, data, target):
    """
    Evaluate process of classification.
    
    Parameters
    ----------
    model : object
        Classificator to be evaluated.
    data : np.array
        Array of values for all calculated leafs properties.
    target : np.array
        Array of numbers which corresponds to leafs species.
    """
    random_forest_lst = []
    
    indices = np.random.permutation(len(data))
    train_X = data[indices[:-100]]
    train_Y = target[indices[:-100]]
    test_X = data[indices[-100:]]
    test_Y = target[indices[-100:]]

    model.fit(train_X, train_Y)

    random_forest_lst.append((len(test_X) - count_mistakes(model.predict(test_X), test_Y)) / len(test_X))

    print('Correct percent: ' + str(round(np.mean(random_forest_lst), 2)) + '%')

In [22]:
evaluate(rf_random.best_estimator_, data, target)

Correct percent: 0.77%


In [23]:
evaluate(random_forest_clf, data, target)

Correct percent: 0.85%


In [24]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [10, 20, 40, 100],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [2, 4, 6],
    'n_estimators': [500, 1000, 1500, 2000]
}

In [25]:
rf = RandomForestClassifier()
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

In [26]:
grid_search.fit(data, target)

Fitting 3 folds for each of 144 candidates, totalling 432 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [10, 20, 40, 100],
                         'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [2, 4, 6],
                         'n_estimators': [500, 1000, 1500, 2000]},
             verbose=2)

In [27]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 100,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 500}

In [28]:
evaluate(grid_search.best_estimator_, data, target)

Correct percent: 0.74%


In [29]:
evaluate(random_forest_clf, data, target)

Correct percent: 0.81%


In [30]:
dump(grid_search.best_estimator_, 'clf_file.pkl')

['clf_file.pkl']

In [31]:
rf = load('clf_file.pkl')

In [32]:
evaluate(rf, data, target)

Correct percent: 0.86%
